<a href="https://colab.research.google.com/github/DevFreak-ui/MMAD/blob/main/NLLB200_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



| **Name**     | **Languages** |
|--------------|---------------|
| Prince       | `['yor_Latn', 'afr_Latn', 'kam_Latn', 'kab_Latn', 'fuv_Latn', 'kon_Latn']` |
| Mardiyyah    | `['amh_Ethi, 'aka_Latn', 'kmb_Latn', 'kin_Latn', 'bem_Latn','arz_Arab']` |
| Faatimah     | `['ibo_Latn', 'hau_Latn', 'ewe_Latn', 'dyu_Latn', 'cjk_Latn','ary_Arab']` |
| Oluwatosin   | `['dik_Latn', 'lin_Latn', 'lua_Latn', 'lug_Latn', 'kik_Latn','knc_Latn']` |
| Mahi         | Running translations with Azure Translate |


###

Language Dictionary:

    "eng_Latn": "English"
    "yor_Latn": "Yoruba"
    "afr_Latn": "Afrikaans"
    "kam_Latn": "Kamba"
    "kab_Latn": "Kabyle"
    "arz_Arab": "Egyptian Arabic"
    "ary_Arab": "Moroccan Arabic"
    "amh_Ethi": "Amharic"
    "aka_Latn": "Akan"
    "kmb_Latn": "Kimbundu"
    "kin_Latn": "Kinyarwanda"
    "ibo_Latn": "Igbo"
    "hau_Latn": "Hausa"
    "ewe_Latn": "Ewe"
    "dyu_Latn": "Dyula"
    "dik_Latn": "Rek"
    "lin_Latn": "Lingala"
    "lua_Latn": "LubaKasai"
    "lug_Latn": "Ganda"
    "bem_Latn": "Bemba"
    "cjk_Latn": "Chokwe"
    "fuv_Latn": "Nigerian Fulfulde"
    "kik_Latn": "Kikuyu"
    "knc_Latn": "Central Kanuri - Latin"
    "kon_Latn": "Kikongo"

In [ ]:
dataset_path = '/content/drive/MyDrive/flickr8k_best_captions.csv'

# TARGET LANGUAGES --> Replace with your list of languages to run
target_langs = ['lang_1', 'lang_2', ...] # Replace this list

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Read file
df = pd.read_csv(dataset_path, header=0)
df.head()

,image,best_caption
0,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
1,1001773457_577c3a7d70.jpg,A black dog and a tri-colored dog playing with...
2,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front o...
3,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
4,1007129816_e794419615.jpg,A man wears an orange hat and glasses .


In [ ]:
batch_size = 150
translated_captions = []
total_batches = (len(df) + batch_size - 1) // batch_size

In [ ]:
# Loop through each target language first
for lang in target_langs:
    print(f"Processing translations for {lang}...")

    total_batches = (len(df) + batch_size - 1) // batch_size  # Calculate total number of batches

    # Loop through the DataFrame in chunks
    for batch_number, start_idx in enumerate(range(0, len(df), batch_size), start=1):
        end_idx = min(start_idx + batch_size, len(df))
        batch_df = df.iloc[start_idx:end_idx]

        print(f"batch {batch_number}/{total_batches}")

        # Tokenize the captions in the current batch
        captions = batch_df['best_caption'].tolist()
        model_inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=True)

        # Translate the batch of captions to the current language
        translated_tokens = model.generate(
            **model_inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids(lang), max_length=40
        )

        # Decode the translated captions
        translated_texts = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

        # Append the results to the list
        for i, translated_text in enumerate(translated_texts):
            translated_captions.append({
                "image": batch_df.iloc[i]['image'],
                "eng_Latn": captions[i],
                lang: translated_text
            })

    # Convert the list of dictionaries to a DataFrame
    df_translated = pd.DataFrame(translated_captions)

    # Save the DataFrame to a CSV file
    df_translated.to_csv(f'eng_{lang}_translated_captions.csv', index=False)

print(f"Completed translations for {len(target_langs)} languages.")